In [1]:
import pickle

from keras.models import Model
from keras.layers import Dense, Flatten, Reshape, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam

import numpy as np
from sklearn.preprocessing import LabelBinarizer

Using TensorFlow backend.


In [9]:
training_file = './data/train.p'
validation_file = './data/valid.p'
testing_file = './data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [10]:
#X_train = (X_train-128)/128
#X_valid = (X_valid-128)/128
#X_test = (X_test-128)/128

In [11]:
lb = LabelBinarizer().fit(np.hstack((y_train, y_valid, y_test)))
y_train = lb.transform(y_train)
y_valid = lb.transform(y_valid)
y_test = lb.transform(y_test)

In [12]:
_input = Input(shape=(32, 32, 3))
conv1 = Conv2D(filters=6, kernel_size=(5, 5), activation='relu', 
               strides=(1, 1), padding='valid')(_input)
pool1 = MaxPooling2D(pool_size=2, strides=2, padding='same')(conv1)
conv2 = Conv2D(filters=16, kernel_size=(5, 5), activation='relu',
               strides=(1, 1), padding='valid')(pool1)
pool2 = MaxPooling2D(pool_size=2, strides=2, padding='same')(conv2)
flattened = Flatten()(pool2)
fc1 = Dense(120, activation='relu')(flattened)
fc2 = Dense(84, activation='relu')(fc1)
output = Dense(43, activation='softmax')(fc2)

In [13]:
model = Model(inputs=[_input], outputs=[output])
model.compile(optimizer=Adam(0.001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 6)         456       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 120)               48120     
__________

In [ ]:
model.fit(X_train, y_train, epochs=50, batch_size=256, 
          validation_data=(X_valid, y_valid))

Train on 34799 samples, validate on 4410 samples
Epoch 1/50
34799/34799 [==============================] - 17s - loss: 2.7355 - acc: 0.4371 - val_loss: 1.3081 - val_acc: 0.6608